<a href="https://colab.research.google.com/github/hmezer/dai-project/blob/main/dai_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/hmezer/dai-project

Cloning into 'dai-project'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 40 (delta 14), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 19.52 MiB | 23.57 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [18]:
import gzip
import json
import glob
import os

# Path to the sample-data directory
data_dir = "dai-project/sample-data"
json_list = []

# Find all .jsonl.gz files in the directory
for file_path in glob.glob(os.path.join(data_dir, "*.jsonl.gz")):
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            json_list.append(json.loads(line))

# Now json_list contains all the JSON objects from the .jsonl.gz files
print(f"Loaded {len(json_list)} JSON objects from .jsonl.gz files.")

# Print the first JSON object
print(json.dumps(json_list[0], indent=2))

Loaded 5059 JSON objects from .jsonl.gz files.
{
  "ResultId": "urn:contentItem:6B6W-5W61-JBG1-804K-00000-00",
  "Date": "2024-01-29T00:00:00Z",
  "Title": "ELS Reports Fourth Quarter Results Continued Strong Performance; Provides 2024 Guidance and Increases Annual Dividend",
  "Document": {
    "DocumentId": "/shared/document/news/urn:contentItem:6B6W-5W61-JBG1-804K-00000-00",
    "DocumentIdType": "DocFullPath",
    "Content": "<entry xmlns=\"http://www.w3.org/2005/Atom\"><id>urn:contentItem:6B6W-5W61-JBG1-804K-00000-00</id><title>ELS Reports Fourth Quarter Results</title><published>2024-01-29T00:00:00Z</published><updated>2025-06-11T07:28:45Z</updated><author><name>LexisNexis</name></author><content type=\"application/xml\"><!--Transformation version 1.25--><articleDoc xmlns=\"\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:noNamespaceSchemaLocation=\"http://www.lexisnexis.com/xmlschemas/content/public/articledoc/1/\" schemaVersion=\"1.8\" xml:lang=\"en\"><articleDocH

In [43]:
from bs4 import BeautifulSoup
import spacy
import string
from spacy.lang.en import English
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

spacy_nlp = spacy.load('en_core_web_sm')

# Load the FinBERT model and tokenizer inside the function
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_nlp = pipeline("sentiment-analysis",
                         model=model,
                         tokenizer=tokenizer,
                         truncation=True,
                         max_length=512)

Device set to use cpu


In [70]:
def analyze_sentiment_from_text(text):
    """
    Analyzes the sentiment of a given text string using FinBERT.

    Args:
        text (str): The input text string.

    Returns:
        tuple: A tuple containing:
            - list: Sentiment analysis results for the lemmatized text.
            - list: Sentiment analysis results for each sentence in the text.
    """

    # Create a list of sentences
    doc = spacy_nlp(text)
    sentences = [sent.text for sent in doc.sents]

    _="""
    import re

    def is_relevant_sentence(sentence):
        # Regex for date/time patterns
        metadata_patterns = [
            r"\bEastern Time\b",
            r"\b\d{4}\b",                         # Year
            r"\bJanuary|\bFebruary|\bMarch|\bApril|\bMay|\bJune|\bJuly|\bAugust|\bSeptember|\bOctober|\bNovember|\bDecember\b",
            r"\b\d{1,2}:\d{2}\s*(AM|PM)?\b",      # Time
        ]
        for pattern in metadata_patterns:
            if re.search(pattern, sentence):
                print(sentence)
                return False
        # Optionally: filter very short sentences
        if len(sentence.split()) < 4:
            return False
        return True

    sentences = [s for s in sentences if is_relevant_sentence(s)]
    """

    sentences_sentiment = sentiment_nlp(sentences)

    return sentences, sentences_sentiment

<>:25: SyntaxWarning: invalid escape sequence '\d'
<>:25: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-2871948619.py:25: SyntaxWarning: invalid escape sequence '\d'
  r"\b\d{4}\b",                         # Year


In [71]:
def get_sentiment_scores(sentiment_scores):
    flag_sum = 0
    num_sentences = len(sentiment_scores)
    num_polarized = 0

    for s in sentiment_scores:
        if s['label'] == 'Positive':
            flag_sum += s['score']
            num_polarized += 1
        elif s['label'] == 'Negative':
            flag_sum -= s['score']
            num_polarized += 1
        # Neutral is ignored for polarized count

    score_including_neutrals = flag_sum / num_sentences if num_sentences else 0
    score_polarized_only = flag_sum / num_polarized if num_polarized else 0

    return score_including_neutrals, score_polarized_only

In [72]:
html_str = json_list[100]["Document"]["Content"]
text = ""
if html_str:
    soup = BeautifulSoup(html_str, 'html.parser')
    body_tag = soup.find('nitf:body.content')
    if body_tag:
        text = body_tag.get_text()

if text:
    def replace_bullet_with_dot(text):
        import re
        # Replace "• " (bullet with space after) with "."
        text = re.sub(r'•\s+', '.', text)
        # Replace any remaining bullet (sticky case) with ". "
        text = re.sub(r'•', '. ', text)
        return text
    text = replace_bullet_with_dot(text)

    sentences, sentences_sentiment = analyze_sentiment_from_text(text)

    print(get_sentiment_scores(sentences_sentiment))
    print("\nSentiment Analysis per Sentence:")
    i = 0
    for sentence in sentences:
        print(f"Sentence {i+1}: {sentence}")
        print(f"Sentiment: {sentences_sentiment[i]['label']}, Score: {sentences_sentiment[i]['score']:.4f}")
        print("-" * 20)
        i += 1

(-7.110834121704102e-05, -0.0002311021089553833)

Sentiment Analysis per Sentence:
Sentence 1: Cohu, Inc. (NASDAQ: COHU), a global supplier of equipment and services optimizing semiconductor manufacturing yield and productivity, today provided a business update:.
Sentiment: Neutral, Score: 0.5118
--------------------
Sentence 2: Cohu expects fourth quarter 2023 revenue in-line with guidance at approximately $137 million.
Sentiment: Neutral, Score: 1.0000
--------------------
Sentence 3: Demand for test and inspection equipment remained soft through fourth quarter 2023 and has continued in first quarter 2024.
Sentiment: Negative, Score: 1.0000
--------------------
Sentence 4: Cohu expects first quarter 2024 revenue to be approximately 20% lower than fourth quarter 2023"We expect fourth quarter revenue to be within our guidance range; however, estimated equipment utilization dropped another 200 bps to 71% at the end of fourth quarter.
Sentiment: Negative, Score: 1.0000
------------------